# You suck at shell scripting!

You do! You're aweful! You are as bad at shell scripting
as Helen Zille is at Twitter. However clever that 
140 character bash one liner seemed at the time,
you know that you regretted it once it became an
integral part of your production build pipeline. This
talk will show you how to write maintainable
Command Line Interfaces with click and pathlib
that won't leave you trying to rewrite your git commit
history after the next production server outage.


## Aim

  * demonstrate
      * pathlib
      * click
  * worked example for file deduplication

## shell scripts

  * Provide some examples of badly written shell scripts
  * Why I've resisted using Python for shell scripting.
      * os.path

## pathlib

Path handling the right way

In [1]:
from pathlib import Path
p = Path('.')
p

WindowsPath('.')

In [2]:
print(str(p))
print(p.absolute())

.
C:\Users\Tobias\Documents\Python Scripts\ctpug-20170701


In [3]:
p = p.absolute()
print(p.as_posix())

C:/Users/Tobias/Documents/Python Scripts/ctpug-20170701


In [4]:
print(p.as_uri())

file:///C:/Users/Tobias/Documents/Python%20Scripts/ctpug-20170701


In [5]:
p.is_dir()

True

In [6]:
p.is_file()

False

In [7]:
p.parent

WindowsPath('C:/Users/Tobias/Documents/Python Scripts')

In [8]:
p.relative_to(p.parent)

WindowsPath('ctpug-20170701')

In [9]:
# subdirectories
[x for x in p.iterdir() if x.is_dir()]

[WindowsPath('C:/Users/Tobias/Documents/Python Scripts/ctpug-20170701/.ipynb_checkpoints')]

In [10]:
# files
[x for x in p.iterdir() if x.is_file()]

[WindowsPath('C:/Users/Tobias/Documents/Python Scripts/ctpug-20170701/hello.py'),
 WindowsPath('C:/Users/Tobias/Documents/Python Scripts/ctpug-20170701/You suck at shell scripting.ipynb')]

In [11]:
list(p.rglob('*.ipynb'))

[WindowsPath('C:/Users/Tobias/Documents/Python Scripts/ctpug-20170701/You suck at shell scripting.ipynb'),
 WindowsPath('C:/Users/Tobias/Documents/Python Scripts/ctpug-20170701/.ipynb_checkpoints/You suck at shell scripting-checkpoint.ipynb')]

In [12]:
p = Path('.') / 'newdir'
p

WindowsPath('newdir')

In [13]:
p.exists()

False

In [14]:
p.mkdir()

In [15]:
p.exists()

True

In [16]:
q = p / 'newfile.txt'
q

WindowsPath('newdir/newfile.txt')

In [17]:
with q.open('wt') as f:
    f.write('The quick brown fox jumped over the lazy dog.')

In [18]:
q.exists() and q.is_file()

True

In [19]:
q.read_text()

'The quick brown fox jumped over the lazy dog.'

## Click (Command Line Interface Creation Kit)

Beautiful CLI APIs
  * automatic help page generation
  * parameter validation
  * arbitrary nesting of commands
  * supports lazy loading of subcommands at runtime

### To cover
  
* arguments vs options
* parameter types

### Basic example

In [ ]:
# %load hello.py
import click

@click.command()
@click.option('--count', default=1, help='Number of greetings.')
@click.option('--name', prompt='Your name', help='The person to greet.')
def hello(count, name):
    """Simple program that greets NAME for a total of COUNT times."""
    for x in range(count):
        click.echo('Hello {}!'.format(name))

if __name__=='__main__':
    hello()


In [29]:
from hashlib import md5, sha1

In [30]:
myhash = locals()['md5']

In [39]:
h = myhash(q.read_bytes())

<function _hashlib.openssl_md5>